In [76]:
from os.path import expanduser
import pandas as pd
import simplejson
import numpy as np
import urllib

In [2]:
# get the google api key
home = expanduser("~")
with open(home + '/.googleapi') as infile:
    apikey = infile.next().strip()

In [6]:
# read in the metadata file
meta_hdf_path = '/Users/bohr/workspace/capstone/DSE_Cohort2_Traffic_Capstone/C2 Exploratory/meta_2015.hdf'
meta_frame = pd.read_hdf(meta_hdf_path, 'meta_2015')
meta_frame.head()

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
584,1113072,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,0.452,ML,2,Black Mountain Rd,314,NaN,NaN,NaN,2015_01_01
585,1113073,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,OR,3,BLACK MOUNTAIN RD,314,NaN,NaN,NaN,2015_01_01
665,1113680,56,E,11,73,66000.0,6.862,7.364,32.953394,-117.133404,0.999,ML,3,BLACK MOUNTAIN RD,434,NaN,NaN,NaN,2015_01_01
666,1113683,56,W,11,73,66000.0,7.383,7.885,32.955202,-117.124689,NaN,FR,2,BLK MOUNTAIN - WB 56,314,NaN,NaN,NaN,2015_01_01
1034,1119041,125,N,11,73,NaN,1.433,3.282,32.608637,-116.967247,0.792,ML,3,1 MI S/O BIRCH RD,19106,NaN,NaN,NaN,2015_01_01


In [81]:
meta_frame.count()

ID           1541
Fwy          1541
Dir          1541
District     1541
County       1541
City         1318
State_PM     1541
Abs_PM       1541
Latitude     1539
Longitude    1539
Length        932
Type         1541
Lanes        1541
Name         1541
User_ID_1    1541
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1541
dtype: int64

In [103]:
meta_frame[meta_frame.Latitude.isnull()]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date
767,1114649,805,S,11,73,66000.0,28.811,28.662,NaN,NaN,0.651,ML,4,S/B AT JCT I-5,4045,NaN,NaN,NaN,2015_12_17
1324,1125383,52,W,11,73,70224.0,14.756,14.756,NaN,NaN,0.695,ML,2,52 WB from 125 Conn,43812,NaN,NaN,NaN,2015_12_17


In [215]:
meta_frame_no_nan = meta_frame.dropna(axis='index', subset=['Latitude', 'Longitude'])
meta_frame_no_nan.index = np.arange(0, len(meta_frame_no_nan))
meta_frame_no_nan['lat_lon'] = meta_frame_no_nan['Latitude'].map(str) + ',' + meta_frame_no_nan['Longitude'].map(str)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [216]:
# spot check
meta_frame_no_nan[meta_frame_no_nan.ID == 1114649]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date,lat_lon


In [217]:
meta_frame_no_nan.count()

ID           1539
Fwy          1539
Dir          1539
District     1539
County       1539
City         1316
State_PM     1539
Abs_PM       1539
Latitude     1539
Longitude    1539
Length        930
Type         1539
Lanes        1539
Name         1539
User_ID_1    1539
User_ID_2       0
User_ID_3       0
User_ID_4       0
file_date    1539
lat_lon      1539
dtype: int64

In [235]:
ELEVATION_BASE_URL = 'https://maps.googleapis.com/maps/api/elevation/json'
def getElevation(lat_lon_list=None, **elvtn_args):
    """
    This function will get the elevation from the google api
    """
    result = None
    elvtn_args.update({
        'locations': "|".join(lat_lon_list),
        'key': apikey
    })

    url = ELEVATION_BASE_URL + '?' + urllib.urlencode(elvtn_args)
    response = simplejson.load(urllib.urlopen(url))

    if 'status' not in response or response['status'] != 'OK':
        print "bad response"
    else:
        result = [item['elevation'] for item in response['results']]
    return result

In [236]:
# # chunk the data into parts of 50
# id_master = []
# elevation_master = []

# chunk = 50
# for item in range(0, 31):
#     start = item*chunk
#     end = (item+1)*chunk
    
#     id_master += list(meta_frame_no_nan['ID'][start:end].values)
#     elevation_master += getElevation(list(meta_frame_no_nan['lat_lon'][start:end]))
    
# print len(id_master)
# print len(elevation_master)

1539
1539


In [237]:
elev_df = pd.DataFrame({'ID': id_master, 'elevation': elevation_master})

In [241]:
meta_frame.columns

Index([u'ID', u'Fwy', u'Dir', u'District', u'County', u'City', u'State_PM',
       u'Abs_PM', u'Latitude', u'Longitude', u'Length', u'Type', u'Lanes',
       u'Name', u'User_ID_1', u'User_ID_2', u'User_ID_3', u'User_ID_4',
       u'file_date'],
      dtype='object')

In [242]:
elev_df.columns

Index([u'ID', u'elevation'], dtype='object')

In [251]:
new_meta_frame = pd.merge(meta_frame, elev_df, how='left')

In [253]:
new_meta_frame[new_meta_frame.elevation.isnull()]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,file_date,elevation
884,1114649,805,S,11,73,66000.0,28.811,28.662,NaN,NaN,0.651,ML,4,S/B AT JCT I-5,4045,NaN,NaN,NaN,2015_12_17,NaN
1441,1125383,52,W,11,73,70224.0,14.756,14.756,NaN,NaN,0.695,ML,2,52 WB from 125 Conn,43812,NaN,NaN,NaN,2015_12_17,NaN


In [254]:
new_meta_frame.to_csv('meta_2015_with_elev.csv')